In [1]:
import torch

In [2]:
def get_data(path):
    with open(path, 'r') as f:
        data = f.readlines()[3:]
        data = [float(data[i].strip()) for i in range(len(data))]
    return data

In [4]:
testdata = []

for i in range(10):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--0000' + str(i) + '.csv'))
for i in range(90):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--000' + str(i+10) + '.csv'))
for i in range(900):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--00' + str(i+100) + '.csv'))
for i in range(9000):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--0' + str(i+1000) + '.csv'))
for i in range(9243):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--' + str(i+10000) + '.csv'))

testdata = torch.tensor(testdata)

In [4]:
def normalize(signal):
    peak = min(signal)
    norm = signal
    for i in range(len(signal)):
        norm[i] = signal[i]/peak
    return norm

def writedata(signal, filename):
    with open('norm/' + filename, 'a') as f:
        for i in range(len(signal)):
            print(signal[i], file = f)

In [5]:
import time
tstart = time.time()
testdata = []
testdata_norm = []

for i in range(10):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--0000' + str(i) + '.csv'))
for i in range(90):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--000' + str(i+10) + '.csv'))
for i in range(900):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--00' + str(i+100) + '.csv'))
for i in range(9000):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--0' + str(i+1000) + '.csv'))
for i in range(9243):
    testdata.append(get_data(r'D:\\My Files\\核技术专题研讨\\Anode\\Anode\\c3--' + str(i+10000) + '.csv'))

for i in range(len(testdata)):
    testdata_norm.append(normalize(testdata[i]))
    writedata(testdata_norm[i], str(i) + '.txt')

testdata = torch.tensor(testdata)
tend = time.time()
print(tend - tstart)

41.51531219482422


In [6]:
testdata_norm = torch.tensor(testdata_norm)

K-means model

In [9]:
class KMEANS:
    def __init__(self, n_clusters = 2, max_iter = None, verbose = True, device = torch.device("cpu")):

        self.n_cluster = n_clusters
        self.n_clusters = n_clusters
        self.labels = None
        self.dists = None  # shape: [x.shape[0],n_cluster]
        self.centers = None
        self.variation = torch.Tensor([float("Inf")]).to(device)
        self.verbose = verbose
        self.started = False
        self.representative_samples = None
        self.max_iter = max_iter
        self.count = 0
        self.device = device

    def fit(self, x):
        init_row = torch.randint(0, x.shape[0], (self.n_clusters,)).to(self.device)
        init_points = x[init_row]
        self.centers = init_points
        while True:
            self.nearest_center(x)
            self.update_center(x)
            if self.verbose:
                print(self.variation, torch.argmin(self.dists, (0)))
            if torch.abs(self.variation) < 1e-6 and self.max_iter is None:
                break
            elif self.max_iter is not None and self.count == self.max_iter:
                break

            self.count += 1

        self.representative_sample()

        with open('kmeans.txt', 'a') as f:
            for i in range(x.shape[0]):
                print(self.dists[i][0].item(), self.dists[i][1].item(), file = f)

    def nearest_center(self, x):
        labels = torch.empty((x.shape[0],)).long().to(self.device)
        dists = torch.empty((0, self.n_clusters)).to(self.device)
        for i, sample in enumerate(x):
            dist = torch.sum(torch.mul(sample - self.centers, sample - self.centers), (1))
            labels[i] = torch.argmin(dist)
            dists = torch.cat([dists, dist.unsqueeze(0)], (0))
        self.labels = labels
        if self.started:
            self.variation = torch.sum(self.dists - dists)
        self.dists = dists
        self.started = True

    def update_center(self, x):
        centers = torch.empty((0, x.shape[1])).to(self.device)
        for i in range(self.n_clusters):
            mask = self.labels == i
            cluster_samples = x[mask]
            centers = torch.cat([centers, torch.mean(cluster_samples, (0)).unsqueeze(0)], (0))
        self.centers = centers

    def representative_sample(self):
        self.representative_samples = torch.argmin(self.dists, (0))

In [11]:
kmeans = KMEANS()
kmeans.fit(testdata)

tensor([inf]) tensor([10561,   987])
tensor(-4727.5620) tensor([16569, 17123])
tensor(3383.4102) tensor([11479,  1791])
tensor(1715.5330) tensor([2849, 1791])
tensor(823.8400) tensor([2849, 8119])
tensor(432.7294) tensor([2849, 8119])
tensor(190.7943) tensor([2849, 8119])
tensor(137.4166) tensor([2849, 8119])
tensor(77.2113) tensor([2849, 8119])
tensor(50.9358) tensor([2849, 8119])
tensor(27.0934) tensor([2849, 8119])
tensor(9.0110) tensor([2849, 8119])
tensor(5.4012) tensor([2849, 8119])
tensor(5.3956) tensor([2849, 8119])
tensor(1.7976) tensor([2849, 8119])
tensor(5.3868) tensor([2849, 8119])
tensor(10.7429) tensor([2849, 8119])
tensor(7.1441) tensor([2849, 8119])
tensor(1.7857) tensor([2849, 8119])
tensor(1.7848) tensor([2849, 8119])
tensor(0.) tensor([2849, 8119])


In [36]:
def PCA_svd(X, k, center=True):
  n = X.size()[0]
  ones = torch.ones(n).view([n,1])
  h = ((1/n) * torch.mm(ones, ones.t())) if center else torch.zeros(n*n).view([n,n])
  H = torch.eye(n) - h
  #H = H.cuda()
  X_center =  torch.mm(H.double(), X.double())
  u, s, v = torch.svd(X_center)
  components  = v[:k].t()
  #explained_variance = torch.mul(s[:k], s[:k])/(n-1)
  return components